In [1]:
from mac import cbc_mac
from aes import aes_ecb_decrypt
from utils import xor
pt = "alert('MZA who was that?');\n"
key = b"YELLOW SUBMARINE"
iv = b"\x00" * 16
hash = cbc_mac(pt.encode(), key, iv).hex()
print('hashing verified?', hash == "296b8d7cb78a243dda4d0a61d33bbdd1")

hashing verified? True


In [2]:
# // at the end to add a javascript comment
alert_msg = b"alert('Ayo, the Wu is back!');//"
print(f'length of alert_msg: {len(alert_msg)}')
wanted_hashbytes = bytes.fromhex(hash)
print(f'wanted last block: {wanted_hashbytes}')

length of alert_msg: 32
wanted last block: b')k\x8d|\xb7\x8a$=\xdaM\na\xd3;\xbd\xd1'


<img src="https://www.researchgate.net/profile/Rhouma-Rhouma/publication/215783767/figure/fig1/AS:394138559238144@1470981363092/Cipher-block-chaining-CBC-mode-encryption.png">   

Since we know what the last byte we want is, and we can find out the ciphertext of the previous step (we control the pt), we can decrypt the wanted last block with the key we know in ECB mode and then xor it with the ciphertext of previous step to get the plaintext we want.  

In [3]:
from aes import aes_cbc_encrypt
from padding import pkcs7_pad
from utils import get_blocks


print(f'wanted last block:\t{wanted_hashbytes}')
dec_last_block = aes_ecb_decrypt(wanted_hashbytes, key)
print(f'decrypted last block:\t{dec_last_block}')
forged_pt_last_block = b'\x10' * 16
print(f'forged last block:\t{forged_pt_last_block}')
ct_prev_block = xor(dec_last_block, forged_pt_last_block)
print(f'ct_prev_block:\t\t{ct_prev_block}')

wanted last block:	b')k\x8d|\xb7\x8a$=\xdaM\na\xd3;\xbd\xd1'
decrypted last block:	b'\xf2#o\x15\xf3\xabQk^\x1c|\xc5\x05\x16\xe9\x8e'
forged last block:	b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'
ct_prev_block:		b'\xe23\x7f\x05\xe3\xbbA{N\x0cl\xd5\x15\x06\xf9\x9e'


The structure is 4 blocks.  
[block 1 : alert_msg0] - [block 2 : alert_msg1] - [block 3 : WANTED] - [block 4 : b'\x10'*16]  
to get the WANTED pt, we decrypt ct_prev_block and xor it with encryption of alert_msg1 in cbc mode with no padding.

In [4]:
enc_last_block_alert_msg = get_blocks(aes_cbc_encrypt(alert_msg, key, iv))[1]
dec_prev_block = aes_ecb_decrypt(ct_prev_block, key)
forged_pt_prev_block = xor(dec_prev_block, enc_last_block_alert_msg)
print(f'forged prev block:\t{forged_pt_prev_block}')
forged_msg = alert_msg + forged_pt_prev_block
print(f'forged msg:\t\t{forged_msg}')
forged_hash = cbc_mac(forged_msg, key, iv).hex()
print(f'forged hash:\t\t{forged_hash}')
print('hashing verified?', forged_hash == hash)

forged prev block:	b'\xb0\xdf\x93;\xe6\x8fW\xf7\xfc\xdc\xa0\x04\xe2\x1b\xd5\xa6'
forged msg:		b"alert('Ayo, the Wu is back!');//\xb0\xdf\x93;\xe6\x8fW\xf7\xfc\xdc\xa0\x04\xe2\x1b\xd5\xa6"
forged hash:		296b8d7cb78a243dda4d0a61d33bbdd1
hashing verified? True


In [6]:
def cbc_mac_hash_forgery(orig_hash: str, forged_msg: bytes, key: bytes, iv: bytes):
    wanted_hashbytes = bytes.fromhex(orig_hash)
    if len(forged_msg) % 16 != 0:
        raise ValueError('forged_msg must be a multiple of 16 bytes')
    dec_last_block = aes_ecb_decrypt(wanted_hashbytes, key)
    forged_pt_last_block = b'\x10' * 16
    ct_prev_block = xor(dec_last_block, forged_pt_last_block)
    enc_last_block_msg = get_blocks(aes_cbc_encrypt(forged_msg, key, iv))[-1]
    dec_prev_block = aes_ecb_decrypt(ct_prev_block, key)
    forged_pt_prev_block = xor(dec_prev_block, enc_last_block_msg)
    forged_msg = forged_msg + forged_pt_prev_block
    forged_hash = cbc_mac(forged_msg, key, iv).hex()
    return forged_hash, forged_msg

orig_hash = "296b8d7cb78a243dda4d0a61d33bbdd1"
forged_msg = b"alert('Ayo, the Wu is back!');//"
forged_hash, forged_msg = cbc_mac_hash_forgery(orig_hash, forged_msg, key, iv)
print(f'forged hash:\t\t{forged_hash}')
print(f'forged msg:\t\t{forged_msg}')
print('hashing verified?', forged_hash == hash)

forged hash:		296b8d7cb78a243dda4d0a61d33bbdd1
forged msg:		b"alert('Ayo, the Wu is back!');//\xb0\xdf\x93;\xe6\x8fW\xf7\xfc\xdc\xa0\x04\xe2\x1b\xd5\xa6"
hashing verified? True
